In [11]:
import os
from datetime import datetime
import pandas as pd
import yfinance as yf
import csv

In [12]:
def update_stock_csv_by_years(tickers, years, frequency='daily', folder='stocks_datasets'):
    interval_map = {'daily': '1d', 'hourly': '1h'}
    if frequency not in interval_map:
        raise ValueError("Frequency must be 'daily' or 'hourly'")

    os.makedirs(folder, exist_ok=True)

    for ticker in tickers:
        all_years_data = []

        # Download year by year
        for year in years:
            interval = interval_map[frequency]
            start_date = f"{year}-01-01"
            end_date = f"{year + 1}-08-01"

            if frequency == 'hourly' and (datetime.today().year - year) > 2:
                print(f"⚠ Hourly data not available before ~2 years ago for {ticker} in {year}. Switching to daily.")
                interval = '1d'

            try:
                print(f"🔄 Fetching {interval} data for {ticker} in {year}...")
                data = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)

                if not data.empty:
                    data.index = pd.to_datetime(data.index)
                    data.index.name = "Date"
                    all_years_data.append(data)
                else:
                    print(f"⚠ No data for {ticker} in {year}")
            except Exception as e:
                print(f"❌ Error with {ticker} in {year}: {e}")

        if all_years_data:
            new_data = pd.concat(all_years_data)
            file_path = os.path.join(folder, f"{ticker}.csv")

            if os.path.exists(file_path):
                try:
                    # Skip custom header rows
                    old_data = pd.read_csv(file_path, skiprows=3, parse_dates=["Date"], index_col="Date")

                    # Flatten MultiIndex if present
                    if isinstance(old_data.columns, pd.MultiIndex):
                        old_data.columns = [str(col[0]) for col in old_data.columns]

                    # Ensure required columns exist
                    required_cols = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
                    for col in required_cols:
                        if col not in old_data.columns:
                            old_data[col] = pd.NA

                except Exception as e:
                    print(f"❌ Could not read old data for {ticker}: {e}")
                    old_data = pd.DataFrame()

                if not old_data.empty:
                    combined = pd.concat([old_data, new_data])
                    combined = combined[~combined.index.duplicated(keep="last")]
                else:
                    combined = new_data
            else:
                combined = new_data

            combined.sort_index(inplace=True)

            # Keep only required columns in order
            cols_order = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
            cols_order_existing = [col for col in cols_order if col in combined.columns]
            df_to_save = combined[cols_order_existing].copy()

            # Rename 'Adj Close' → 'Price'
            df_to_save.rename(columns={'Adj Close': 'Price'}, inplace=True)

            # ✅ Flatten MultiIndex if needed
            if isinstance(df_to_save.columns, pd.MultiIndex):
                df_to_save.columns = [str(col[0]) for col in df_to_save.columns]
            else:
                df_to_save.columns = df_to_save.columns.astype(str)

            # Write CSV in custom format
            with open(file_path, 'w', newline='') as f:
                writer = csv.writer(f)

                # Row 1: Date + column names
                writer.writerow(['Date'] + df_to_save.columns.tolist())

                # Row 2: Ticker repeated
                writer.writerow(['Ticker'] + [ticker] * len(df_to_save.columns))

                # Row 3: Date + empty columns
                writer.writerow(['Date'] + [''] * len(df_to_save.columns))

                # Data rows
                for idx, row in df_to_save.iterrows():
                    writer.writerow([idx.strftime('%Y-%m-%d')] + list(row.values))

            print(f"✅ Updated {ticker}: {file_path}")

In [13]:
update_stock_csv_by_years(
    tickers=["ES=F"],
    years=range(2005, 2025),
    frequency="daily"
)

🔄 Fetching 1d data for ES=F in 2005...


C:\Users\spenc\AppData\Local\Temp\ipykernel_1616\3602911811.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)


🔄 Fetching 1d data for ES=F in 2006...


C:\Users\spenc\AppData\Local\Temp\ipykernel_1616\3602911811.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)


🔄 Fetching 1d data for ES=F in 2007...


...

🔄 Fetching 1d data for ES=F in 2023...


C:\Users\spenc\AppData\Local\Temp\ipykernel_1616\3602911811.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)


🔄 Fetching 1d data for ES=F in 2024...


C:\Users\spenc\AppData\Local\Temp\ipykernel_1616\3602911811.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)


❌ Could not read old data for ES=F: Missing column provided to 'parse_dates': 'Date'
✅ Updated ES=F: stocks_datasets\ES=F.csv
